#1. Install and Import Dependencies

In [1]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [3]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# load the pretrained model
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

# 3. Encode and Calculate Sentiment

In [4]:
# pass a string to tokenize our model
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [18]:
tokens[0]

tensor([  101, 10197, 10140, 12050, 10502, 12296, 10598, 10662, 16197,   119,
        11838,   102])

In [5]:
result = model(tokens)

In [6]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [7]:
int(torch.argmax(result.logits))+1

4

# 4. Collect Reviews

In [8]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [9]:
reviews

["Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 'The food was delicious. The ricotta pancakes were light and tasty.  The cream brûlée French toast was a fruit lovers delight. There was fresh strawberries, blueberries, passion fruit and custard. No syrup required. And crispy kale as a side ordered with eggs. Worth the walk over the Pyrmont bridge.',
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so

# 5. Load Reviews into DataFrame and Score

In [10]:
import numpy as np
import pandas as pd

In [11]:
# Create a DataFrame 'df' from the array 'reviews' with a single column named 'review'
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [12]:
df['review'].iloc[0]

"Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!"

In [13]:
# Function to calculate sentiment score of a given review
def sentiment_score(review):
    # Tokenize the review and obtain tokens
    tokens = tokenizer.encode(review, return_tensors='pt')

    # Pass tokens through the model to get sentiment prediction
    result = model(tokens)

    # Get the index of the maximum value in the logits and convert to integer
    # Shift sentiment score from 0-based to 1-based and return
    return int(torch.argmax(result.logits)) + 1


In [14]:
# Obtain sentiment score for the review at index 1 of the DataFrame 'df'
sentiment_score(df['review'].iloc[1])


4

In [15]:
# Add a new column 'sentiment' to DataFrame 'df' by applying the sentiment_score function
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
df

,review,sentiment
0,Very cute coffee shop and restaurant. They hav...,4
1,The food was delicious. The ricotta pancakes w...,4
2,Six of us met here for breakfast before our wa...,4
3,"Great service, lovely location, and really ama...",5
4,Great place with delicious food and friendly s...,5
5,Some of the best Milkshakes me and my daughter...,5
6,Great food amazing coffee and tea. Short walk ...,5
7,It was ok. Had coffee with my friends. I'm new...,3
8,Ricotta hot cakes! These were so yummy. I ate ...,5
9,We came for brunch twice in our week-long visi...,4


In [17]:
df['review'].iloc[3]

"Great service, lovely location, and really amazing food. Words don't do justice. We had the mushroom parm bruschetta and the loaded double double. Both were so tasty. Also love the Aussie black tea and a flat white. Wish I had more mornings in Sydney to eat breakfast there. Highly recommend."

In [19]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) +1


In [20]:
sentiment_score(df['review'].iloc[1])

4

In [21]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [22]:
df['review']

0    Very cute coffee shop and restaurant. They hav...
1    The food was delicious. The ricotta pancakes w...
2    Six of us met here for breakfast before our wa...
3    Great service, lovely location, and really ama...
4    Great place with delicious food and friendly s...
5    Some of the best Milkshakes me and my daughter...
6    Great food amazing coffee and tea. Short walk ...
7    It was ok. Had coffee with my friends. I'm new...
8    Ricotta hot cakes! These were so yummy. I ate ...
9    We came for brunch twice in our week-long visi...
Name: review, dtype: object

end of project